<a href="https://colab.research.google.com/github/kanade9/AtCoder/blob/master/hoseinlp3_cabocha2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CaboChaによる係り受け解析
## インストール

各種パッケージのインストール．

結構時間がかかる（５分程度）
最語に下記メッセージが出るのを確認．

cabocha of 0.69

In [0]:
!apt install -y \
    curl \
    file \
    git \
    libmecab-dev \
    make \
    mecab \
    mecab-ipadic-utf8 \
    swig \
    xz-utils
!pip install mecab-python3

import os
filename_crfpp = 'crfpp.tar.gz'
!wget "https://drive.google.com/uc?export=download&id=0B4y35FiV1wh7QVR6VXJ5dWExSTQ" -O $filename_crfpp
!tar zxvf $filename_crfpp
%cd CRF++-0.58
!./configure
!make
!make install
%cd ..

os.environ['LD_LIBRARY_PATH'] += ':/usr/local/lib' 

filename_cabocha = 'cabocha.tar.bz2'
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=0B4y35FiV1wh7SDd1Q1dUQkZQaUU' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=0B4y35FiV1wh7SDd1Q1dUQkZQaUU" -O $filename_cabocha && rm -rf /tmp/cookies.txt
!bzip2 -dc $filename_cabocha | tar xvf -
%cd cabocha-0.69
!./configure --with-mecab-config=`which mecab-config` --with-charset=UTF8
!make
!make check
!make install
%cd ..

%cd cabocha-0.69/python
!python setup.py build_ext
!python setup.py install
!ldconfig
%cd ../..

!cabocha --version

Reading package lists... Done
Building dependency tree       
Reading state information... Done
make is already the newest version (4.1-9.1ubuntu1).
xz-utils is already the newest version (5.2.2-1.3).
libmecab-dev is already the newest version (0.996-5).
mecab is already the newest version (0.996-5).
mecab-ipadic-utf8 is already the newest version (2.7.0-20070801+main-1).
swig is already the newest version (3.0.12-1).
curl is already the newest version (7.58.0-2ubuntu3.7).
file is already the newest version (1:5.32-2ubuntu0.2).
git is already the newest version (1:2.17.1-1ubuntu0.4).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
--2019-10-04 04:37:06--  https://drive.google.com/uc?export=download&id=0B4y35FiV1wh7QVR6VXJ5dWExSTQ
Resolving drive.google.com (drive.google.com)... 74.125.142.113, 74.125.142.102, 74.125.142.101, ...
Connecting to drive.google.com (drive.google.com)|74.125.142.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporaril

## 係り受け解析を実行

In [0]:
import CaboCha
c = CaboCha.Parser()

sentence = "一郎は二郎が書いた絵を三郎に送った"
tree =  c.parse(sentence)

# ツリー型の表示
print(tree.toString(CaboCha.FORMAT_TREE))
# XML型の表示
print(tree.toString(CaboCha.FORMAT_XML))

一郎は---------D
  二郎が-D     |
    書いた-D   |
        絵を---D
        三郎に-D
          送った
EOS

<sentence>
 <chunk id="0" link="5" rel="D" score="-1.614596" head="0" func="1">
  <tok id="0" feature="名詞,固有名詞,人名,名,*,*,一郎,イチロウ,イチロー">一郎</tok>
  <tok id="1" feature="助詞,係助詞,*,*,*,*,は,ハ,ワ">は</tok>
 </chunk>
 <chunk id="1" link="2" rel="D" score="1.207924" head="2" func="3">
  <tok id="2" feature="名詞,固有名詞,人名,名,*,*,二郎,ジロウ,ジロー">二郎</tok>
  <tok id="3" feature="助詞,格助詞,一般,*,*,*,が,ガ,ガ">が</tok>
 </chunk>
 <chunk id="2" link="3" rel="D" score="1.541331" head="4" func="5">
  <tok id="4" feature="動詞,自立,*,*,五段・カ行イ音便,連用タ接続,書く,カイ,カイ">書い</tok>
  <tok id="5" feature="助動詞,*,*,*,特殊・タ,基本形,た,タ,タ">た</tok>
 </chunk>
 <chunk id="3" link="5" rel="D" score="-1.614596" head="6" func="7">
  <tok id="6" feature="名詞,一般,*,*,*,*,絵,エ,エ">絵</tok>
  <tok id="7" feature="助詞,格助詞,一般,*,*,*,を,ヲ,ヲ">を</tok>
 </chunk>
 <chunk id="4" link="5" rel="D" score="-1.614596" head="8" func="9">
  <tok id="8" feature="名詞,固有名詞,地域,一般,*,*,三郎,サブロウ,サブロー">

上記の結果について，"id"は文節のID．  "link"は，係り先の文節IDを表す．

私は(0)／猫が(1)／好きな(2)／人と(3)／話した(4)
* 私は(0) → 話した (4)
* 猫が(1) → 好きな (2)
* 好きな(2) → 人と (3)
* 人と(3) → 話した (4)



## XMLを解析する

In [0]:
import CaboCha
c = CaboCha.Parser()

sentence = "私は猫が好きな人と話した．　私は犬が好きです． 私は走った．　あなたは走り続けた．　"
tree =  c.parse(sentence)

import xml.etree.ElementTree as ET

xml = tree.toString(CaboCha.FORMAT_XML)
root = ET.fromstring(xml) # xmlを解析

for chunk in root.findall('chunk'): # 文節毎にループ
  # 例: <chunk id="0" link="4" rel="D" score="-1.678690" head="0" func="1">
  # ここから "id" と "link" の情報を取得
  chunk_id = chunk.attrib['id']
  chunk_link = chunk.attrib['link']
  print('*** chunk id =', chunk_id, '=>', chunk_link)
  
  for token in chunk.findall('tok'):
    # 例: <tok id="0" feature="名詞,代名詞,一般,*,*,*,私,ワタシ,ワタシ">私</tok>
    # 例: <tok id="27" feature="動詞,自立,*,*,五段・ラ行,連用形,走る,ハシリ,ハシリ">走り</tok>
    #
    # ここから， <tok>私</tok>で囲まれた"text"と， "id", 
    # "feature"のカンマ区切りから，品詞と基本形を取得
    token_text = token.text
    token_id = token.attrib['id']
    token_hinshi = token.attrib['feature'].split(',')[0]
    token_kihon = token.attrib['feature'].split(',')[-3]
    print('{} [{}] ({}: {})'.format(token_text, token_kihon, token_id, token_hinshi))
    

*** chunk id = 0 => 4
私 [私] (0: 名詞)
は [は] (1: 助詞)
*** chunk id = 1 => 2
猫 [猫] (2: 名詞)
が [が] (3: 助詞)
*** chunk id = 2 => 3
好き [好き] (4: 名詞)
な [だ] (5: 助動詞)
*** chunk id = 3 => 4
人 [人] (6: 名詞)
と [と] (7: 助詞)
*** chunk id = 4 => 7
話し [話す] (8: 動詞)
た [た] (9: 助動詞)
． [．] (10: 記号)
　 [　] (11: 記号)
*** chunk id = 5 => 7
私 [私] (12: 名詞)
は [は] (13: 助詞)
*** chunk id = 6 => 7
犬 [犬] (14: 名詞)
が [が] (15: 助詞)
*** chunk id = 7 => 11
好き [好き] (16: 名詞)
です [です] (17: 助動詞)
． [．] (18: 記号)
*** chunk id = 8 => 9
私 [私] (19: 名詞)
は [は] (20: 助詞)
*** chunk id = 9 => 11
走っ [走る] (21: 動詞)
た [た] (22: 助動詞)
． [．] (23: 記号)
　 [　] (24: 記号)
*** chunk id = 10 => 11
あなた [あなた] (25: 名詞)
は [は] (26: 助詞)
*** chunk id = 11 => -1
走り [走る] (27: 動詞)
続け [続ける] (28: 動詞)
た [た] (29: 助動詞)
． [．] (30: 記号)
　 [　] (31: 記号)


"私"に関係する文節をあつめてみよう

In [0]:
import CaboCha
c = CaboCha.Parser()

sentence = "私は猫が好きな人と話した．　私は犬が好きです． あなたは猫が好き．　私はタピオカを飲む．　私は走った．"
tree =  c.parse(sentence)

import xml.etree.ElementTree as ET

xml = tree.toString(CaboCha.FORMAT_XML)
root = ET.fromstring(xml) # xmlを解析

target = "私"

id2link = dict() # 係り元 -> 係り先の関係を記憶
target_ids = list() # ターゲットの係り元IDを記憶
id2tokens = dict() # 文節IDごとの形態素リストを記憶

for chunk in root.findall('chunk'): # 文節毎にループ
  chunk_id = chunk.attrib['id']
  chunk_link = chunk.attrib['link']

  # 文節の形態素をtokensに保存しておく
  tokens = []
  for token in chunk.findall('tok'):
    token_text = token.text
    token_hinshi = token.attrib['feature'].split(',')[0]
    token_kihon = token.attrib['feature'].split(',')[-3]
    tokens.append(token_text)

    # ターゲットの形態素が出現したら，文節IDをtarget_idsに保存
    if token_text == target: 
      target_ids.append(chunk_id)
      
  id2link[chunk_id] = chunk_link # ID毎の係り先を保存
  id2tokens[chunk_id] = tokens # ID毎の形態素リストを保存

# 結果を表示
for chunk_id, chunk_link in id2link.items():
  if chunk_id not in target_ids: # 係り元がターゲット文節でなかったら次へ
    continue
  if chunk_link == '-1':  # 係り先が無い
    continue
  print(chunk_id, chunk_link)
  print(' '.join(id2tokens[chunk_id]), '=>', ' '.join(id2tokens[chunk_link]))
  


----------
id2link
{'0': '4', '1': '2', '2': '3', '3': '4', '4': '7', '5': '7', '6': '7', '7': '15', '8': '15', '9': '10', '10': '15', '11': '13', '12': '13', '13': '15', '14': '15', '15': '-1'}
----------
id2tokens
{'0': ['私', 'は'], '1': ['猫', 'が'], '2': ['好き', 'な'], '3': ['人', 'と'], '4': ['話し', 'た', '．', '\u3000'], '5': ['私', 'は'], '6': ['犬', 'が'], '7': ['好き', 'です', '．'], '8': ['あなた', 'は'], '9': ['猫', 'が'], '10': ['好き', '．', '\u3000'], '11': ['私', 'は'], '12': ['タピオカ', 'を'], '13': ['飲む', '．', '\u3000'], '14': ['私', 'は'], '15': ['走っ', 'た', '．']}
----------
target_ids
['0', '5', '11', '14']
----------
0 4
私 は => 話し た ． 　
5 7
私 は => 好き です ．
11 13
私 は => 飲む ． 　
14 15
私 は => 走っ た ．


## 演習1: 「走れメロス」から「メロス」に関係する文節を集めてみよう
青空文庫からテキストをダウンロード

In [0]:
!wget https://www.aozora.gr.jp/cards/000035/files/1567_ruby_4948.zip
!unzip 1567_ruby_4948.zip

この演習では，「メロスが含まれる文節 => 係り先の文節 」　を出力してください．

##出力例


```
*** メロスは激怒した
メロス は => 激怒 し た

*** メロスは両手で老爺のからだをゆすぶって質問を重ねた
メロス は => 重ね た

*** 聞いて、メロスは激怒した
メロス は => 激怒 し た

*** メロスは、単純な男であった
メロス は 、 => 男 で あっ た
```


In [0]:
import CaboCha
import codecs # shift_jisの文字コードのファイルをオープンするため
import xml.etree.ElementTree as ET
import re

parser = CaboCha.Parser()
target = '走る'

with codecs.open("hashire_merosu.txt", 'r', 'shift_jis') as fp:
   for line in fp: #1行ずつ読込
      line = re.sub('《.*》', '', line).strip() # ルビなどの不要部分を除去
      line = re.sub('[「」]', '', line).strip() # かっこを除去
      
      for sent in line.split('。'):
        sent = sent.strip()
        tree = parser.parse(sent) # 係り受け解析
        xml = tree.toString(CaboCha.FORMAT_XML) # XML形式を取得

        if target in sent:
          print()
          print('***', sent)

        root = ET.fromstring(xml) # xmlを解析
        
        ###########################################3
        # ここにプログラムを書いてください
        ###########################################3
   
        id2link = dict()
        id2tokens = dict()
        target_ids = list()
        for chunk in root.findall('chunk'): # 文節毎にループ
          chunk_id = chunk.attrib['id']
          chunk_link = chunk.attrib['link']
          
          tokens = []
          for token in chunk.findall('tok'):
            token_text = token.text
            tokens.append(token_text)
            
            if target == token_text:
              target_ids.append(chunk_id)

          id2link[chunk_id] = chunk_link
          id2tokens[chunk_id] = tokens
         
        #print('-'*10)
        #print(id2link)
        #print(id2tokens)
        #print(target_ids)
        
        for chunk_id, chunk_link in id2link.items():
            #print(chunk_id, chunk_link)
            if chunk_link == '-1':
              continue
            if chunk_link in target_ids:
              print(id2tokens[chunk_id], '->', id2tokens[chunk_link])
             


*** 殺される為に走るのだ
['為', 'に'] -> ['走る', 'の', 'だ']

*** 身代りの友を救う為に走るのだ
['為', 'に'] -> ['走る', 'の', 'だ']

*** 走るのは、やめて下さい

*** 走るより他は無い

*** 走るのは、やめて下さい

*** それだから、走るのだ
['それ', 'だ', 'から', '、'] -> ['走る', 'の', 'だ']

*** 信じられているから走るのだ
['信じ', 'られ', 'て', 'いる', 'から'] -> ['走る', 'の', 'だ']

*** それでは、うんと走るがいい
['それでは', '、'] -> ['走る', 'が', 'いい']
['うんと'] -> ['走る', 'が', 'いい']

*** 走るがいい


## 演習2: 「走れメロス」から「走る」に関係する文節を集めてみよう

この演習では，「係り元の文節 => 「走」が含まれる文節 」　を出力してください．

##出力例


```
*** さて、メロスは、ぶるんと両腕を大きく振って、雨中、矢の如く走り出た
さて 、 => 走り出 た
メロス は 、 => 走り出 た
大きく 振っ て 、 => 走り出 た
如く => 走り出 た

*** 殺される為に走るのだ
為 に => 走る の だ

*** 身代りの友を救う為に走るのだ
為 に => 走る の だ

```



In [0]:
import CaboCha
import codecs # shift_jisの文字コードのファイルをオープンするため
import xml.etree.ElementTree as ET
import re

parser = CaboCha.Parser()
target = '走'

nouns = [] #空の名詞 リスト
with codecs.open("hashire_merosu.txt", 'r', 'shift_jis') as fp:
   for line in fp: #1行ずつ読込
      line = re.sub('《.*》', '', line).strip() # ルビなどの不要部分を除去
      line = re.sub('[「」]', '', line).strip() # かっこを除去
      
      for sent in line.split('。'):
        sent = sent.strip()        
        tree = parser.parse(sent) # 係り受け解析
        xml = tree.toString(CaboCha.FORMAT_XML) # XML形式を取得
        
        if target in sent:
          print()
          print('***', sent)
        root = ET.fromstring(xml) # xmlを解析

  
        ###########################################3
        # ここにプログラムを書いてください
        ###########################################3